In [3]:
import requests as re
from tqdm import tqdm

In [ ]:
USERNAME = 'mdugue@student.ethz.ch'  #### CHANGE FOR USERNAME
PASSWORD = 'ravpex-gUpwod-9fywma' ### CHANGE FOR PASSWORD
URL = 'https://download.dataspace.copernicus.eu/odata/v1/Products(7730bb6f-208b-468a-83dd-2bb6de1194e0)/$value' ### SINGLE FILE TO DOWNLOAD (get the link from the info page of the product on copernicus browser)
FILE = '/run/user/1000/gvfs/smb-share:server=ifu-eo-cds-1.ethz.ch,share=working/Marion/AGRI4SAR/Sentinel-1/S1A_IW_SLC__1SDV_20250303T054335_20250303T054402_058136_072E47_D5E1.SAFE.zip' ### Where to save it

In [9]:
def get_access_token(username: str, password: str) -> str:
    data = {
        "client_id": "cdse-public",
        "username": username,
        "password": password,
        "grant_type": "password",
    }
    try:
        r = re.post(
            "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
            data=data,
            timeout=10,
        )
        r.raise_for_status()
    except Exception as e:
        raise Exception(f'Access token creation failed. Reponse from the server was: {r.json()}') from e
    return r.json()["access_token"]


In [10]:
access_token = get_access_token(USERNAME, PASSWORD)

headers = {"Authorization": f"Bearer {access_token}"}
session = re.Session()
session.headers.update(headers)

response = session.get(URL, headers=headers, stream=True)

total_size = int(response.headers.get('content-length', 0))
block_size = 8192

with open(FILE, "wb") as file:
    for chunk in tqdm(response.iter_content(chunk_size=block_size), total=total_size // block_size, unit='KiB', unit_scale=True):
        if chunk:
            file.write(chunk)


1.01MKiB [15:26, 1.09kKiB/s]                          
